In this project, we will use an RNN architecture to build a Machine Translation model.

It will use as a corpus wikipedia dumps.

Either the source or the target will be English. We will, in our case, try English to French Translation.

In [ ]:
#imports
!pip3 install numpy
!pip3 install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cu117
#or any nightly version so long as pytorch > 1.11 https://pytorch.org/
!pip3 install gensim transformers d2l==1.0.0a1.post0

#In pytorch functional.py, change PILLOW_VERSION to __version__
#there are two places to change

Dataset

In [ ]:
#Test samples location and preprocessing
import os
import torch
from d2l import torch as d2l

class MTFraEng(d2l.DataModule):  #@save
    def _download(self):
        d2l.extract(d2l.download(
            d2l.DATA_URL+'fra-eng.zip', self.root,
            '94646ad1522d915e7b0f9296181140edcf86a4f5'))
        with open(self.root + '/fra-eng/fra.txt', encoding='utf-8') as f:
            return f.read()

@d2l.add_to_class(MTFraEng)  #@save
def _preprocess(self, text):
    # Replace non-breaking space with space
    text = text.replace('\u202f', ' ').replace('\xa0', ' ')
    # Insert space between words and punctuation marks
    no_space = lambda char, prev_char: char in ',.!?' and prev_char != ' '
    out = [' ' + char if i > 0 and no_space(char, text[i - 1]) else char
           for i, char in enumerate(text.lower())]
    return ''.join(out)

@d2l.add_to_class(MTFraEng)  #@save
def _tokenize(self, text, max_examples=None):
    src, tgt = [], []
    for i, line in enumerate(text.split('\n')):
        if max_examples and i > max_examples: break
        parts = line.split('\t')
        if len(parts) == 2:
            # Skip empty tokens
            src.append([t for t in f'{parts[0]} <eos>'.split(' ') if t])  # src.append(EOS_token) ? 
            tgt.append([t for t in f'{parts[1]} <eos>'.split(' ') if t])
    return src, tgt

@d2l.add_to_class(MTFraEng)  #@save
def __init__(self, batch_size, num_steps=15, num_train=162000):
    super(MTFraEng, self).__init__()
    self.save_hyperparameters()
    self.arrays, self.src_vocab, self.tgt_vocab = self._build_arrays(
        self._download())


@d2l.add_to_class(MTFraEng)  #@save
def _build_arrays(self, raw_text, src_vocab=None, tgt_vocab=None):
    def _build_array(sentences, vocab, is_tgt=False):
        pad_or_trim = lambda seq, t: (
            seq[:t] if len(seq) > t else seq + ['<pad>'] * (t - len(seq)))
        sentences = [pad_or_trim(s, self.num_steps) for s in sentences]
        if is_tgt:
            sentences = [['<bos>'] + s for s in sentences]
        if vocab is None:
            vocab = d2l.Vocab(sentences, min_freq=2)
        array = torch.tensor([vocab[s] for s in sentences])
        valid_len = (array != vocab['<pad>']).type(torch.int32).sum(1)
        return array, vocab, valid_len
    src, tgt = self._tokenize(self._preprocess(raw_text),
                              self.num_train)
    src_array, src_vocab, src_valid_len = _build_array(src, src_vocab)
    tgt_array, tgt_vocab, _ = _build_array(tgt, tgt_vocab, True)
    return ((src_array, tgt_array[:,:-1], src_valid_len, tgt_array[:,1:]),
            src_vocab, tgt_vocab)

@d2l.add_to_class(MTFraEng)  #@save
def get_dataloader(self, train):
    idx = slice(0, self.num_train) if train else slice(self.num_train, None)
    return self.get_tensorloader(self.arrays, train, idx)

data = MTFraEng(batch_size=5)
    
src, tgt, src_valid_len, label = next(iter(data.train_dataloader()))
print('source:', src.type(torch.int32))
print('decoder input:', tgt.type(torch.int32))
print('source len excluding pad:', src_valid_len.type(torch.int32))
print('label:', label.type(torch.int32))

@d2l.add_to_class(MTFraEng)  #@save
def build(self, src_sentences, tgt_sentences):
    raw_text = '\n'.join([src + '\t' + tgt for src, tgt in zip(
        src_sentences, tgt_sentences)])
    arrays, _, _ = self._build_arrays(
        raw_text, self.src_vocab, self.tgt_vocab)
    return arrays

#src, tgt, _,  _ = data.build(['hi .'], ['salut .'])
#print('source:', data.src_vocab.to_tokens(src[0].type(torch.int32)))
#print('target:', data.tgt_vocab.to_tokens(tgt[0].type(torch.int32)))

Word embedding

# We will use three different types of word embeddings:
# 1. Word2Vec
# 2. GloVe
# 3. FastText


In [ ]:
"""## Word2Vec"""
import os

# Python program to generate word vectors using Word2Vec
 
# importing all necessary modules
 
import gensim
from gensim.models import Word2Vec

In [ ]:
#keep in mind you have to launch the notebook inside the git folder to make this work (second one)
from inspect import getsourcefile
import sys
print(os.path.dirname(getsourcefile(lambda:0)))
print(sys.path[0])
print(os.path.abspath(sys.argv[0]))

In [ ]:
import numpy as np

def save_split():
    data = MTFraEng(batch_size=5)
    with open("samples/source.txt", "w") as f:
        for i in range(0, data.num_train):
            for word in data.arrays[0][i].numpy() :
                f.write(str(word) + " ")
            f.write("\n")
            
    with open("samples/target.txt", "w") as f:
        for i in range(0, data.num_train):
            for word in data.arrays[1][i].numpy() :
                f.write(str(word) + " ")
            f.write("\n")
            
def load_source():
    return np.loadtxt("samples/source.txt", dtype=str)

def load_target():
    return np.loadtxt("samples/target.txt", dtype=str)

def word_to_token(word, src=True):
    if src :
        return data.src_vocab[word]
    else :
        return data.tgt_vocab[word]

def token_to_word(token, src=True):
    if src :
        return data.src_vocab.to_tokens(token)
    else :
        return data.tgt_vocab.to_tokens(token)

def test_similarity(model, word1, word2, model_name, src=True):
    print("Cosine similarity between '" + word1 + "' and '"+ word2 +"' - " + model_name + " : " + str(model.similarity(word_to_token(word1, src), word_to_token(word2, src))))


In [ ]:
data = MTFraEng(batch_size=5)

In [ ]:
#print a few samples
for i in range(5):
    print(token_to_word(data.arrays[0][i].numpy(), True))

for i in range(5):
    print(token_to_word(data.arrays[1][i].numpy(), False))

In [ ]:
save_split();

In [ ]:

source_text = load_source()
one_line_source = source_text.reshape([np.prod(source_text.shape)])

#format to be accepted by Word2Vec
one_line_source = [str(i).split() for i in one_line_source]

print(one_line_source[:5])
#print in words 
print([token_to_word(int(i[0]), src=True) for i in one_line_source[:5]])



target_text = load_target()
one_line_target = target_text.reshape([np.prod(target_text.shape)])

#format to be accepted by Word2Vec
one_line_target = [str(i).split() for i in one_line_target]

print(one_line_target[:5])
#print in words 
print([token_to_word(int(i[0]), src=False) for i in one_line_target[:5]])

In [ ]:
"""## Word2Vec"""

print(one_line_source[:5])

if not os.path.exists(sys.path[0] + "/models/source_w2v_cbow.txt"):
    # Create CBOW model
    source_w2v_model_cbow = gensim.models.Word2Vec(one_line_source, min_count = 3,
                                vector_size = 100, window = 5)

if not os.path.exists(sys.path[0] + "/models/source_w2v_skip.txt"):
    # Create Skip Gram model
    source_w2v_model_skip = gensim.models.Word2Vec(one_line_source, min_count = 3, vector_size = 100,
                                                window = 5, sg = 1)

In [ ]:
#save the models
if not os.path.exists(sys.path[0] + "/models/source_w2v_cbow.txt"):
    source_w2v_model_cbow.wv.save_word2vec_format(sys.path[0] + "/models/source_w2v_cbow.txt", binary=False)
    
if not os.path.exists(sys.path[0] + "/models/source_w2v_skip.txt"):
    source_w2v_model_skip.wv.save_word2vec_format(sys.path[0] + "/models/source_w2v_skip.txt", binary=False)

In [ ]:
#load the models
source_w2v_model_cbow = gensim.models.KeyedVectors.load_word2vec_format(sys.path[0] + "/models/source_w2v_cbow.txt", binary=False)
source_w2v_model_skip = gensim.models.KeyedVectors.load_word2vec_format(sys.path[0] + "/models/source_w2v_skip.txt", binary=False)

In [ ]:
test_similarity(source_w2v_model_cbow.wv, 'hi', '.', "CBOW")
test_similarity(source_w2v_model_cbow.wv, 'hi', 'go', "CBOW")

test_similarity(source_w2v_model_skip.wv, 'hi', '.', "SkipGram")
test_similarity(source_w2v_model_skip.wv, 'hi', 'go', "SkipGram")

In [ ]:
"""## GloVe"""

# coding: utf-8
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

In [ ]:
#once we have the tokenized file, we can call the glove model

####CALL FROM BASH glove_run.py

In [ ]:
#Only do this once
source_file = sys.path[0] + '\\models\\source_glove.txt'
target_file = sys.path[0] + '\\models\\target_glove.txt'
# Load the model, can take a bit of time
source_glove_model = KeyedVectors.load_word2vec_format(source_file, binary=False, no_header=True)
target_glove_model = KeyedVectors.load_word2vec_format(source_file, binary=False, no_header=True)

In [ ]:

# Test the model
test_similarity(source_glove_model, 'hi', '.', "GloVe", src=True)
test_similarity(source_glove_model, 'hi', 'go', "GloVe", src=True)

test_similarity(target_glove_model, 'bonjour', '.', "GloVe", src=False)
test_similarity(target_glove_model, 'bonjour', 'cours', "GloVe", src=False)

#FastText

In [ ]:
"""## FastText"""
from gensim.models import FastText

#if not saved yet we train it
if not os.path.exists(sys.path[0] + "/models/source_fast.txt"):
    source_fast_model = FastText(vector_size=100, window=5, min_count=3)
    source_fast_model.build_vocab(corpus_file=sys.path[0] + '/models/source_glove.txt')
    source_fast_model.train(corpus_file=sys.path[0] + '/models/source_glove.txt', epochs=10, total_examples=source_fast_model.corpus_count, total_words=source_fast_model.corpus_total_words)


In [ ]:
if not os.path.exists(sys.path[0] + "/models/target_fast.txt"):
    target_fast_model = FastText(vector_size=100, window=5, min_count=3)
    target_fast_model.build_vocab(corpus_file=sys.path[0] + '/models/target_glove.txt')
    target_fast_model.train(corpus_file=sys.path[0] + '/models/target_glove.txt', epochs=10, total_examples=target_fast_model.corpus_count, total_words=target_fast_model.corpus_total_words)

In [ ]:
if not os.path.exists(sys.path[0] + "/models/source_fast.txt"):
    source_fast_model.wv.save_word2vec_format(sys.path[0] + "/models/source_fast.txt", binary=False)
if not os.path.exists(sys.path[0] + "/models/target_fast.txt"):
    target_fast_model.wv.save_word2vec_format(sys.path[0] + "/models/target_fast.txt", binary=False)

In [ ]:
#if saved we load it
source_fast_model = KeyedVectors.load_word2vec_format(sys.path[0] + "/models/source_fast.txt", binary=False, no_header=True)
source_fast_model = KeyedVectors.load_word2vec_format(sys.path[0] + "/models/target_fast.txt", binary=False, no_header=True)

In [ ]:
test_similarity(source_fast_model.wv,'hi', '.', "FastText", src=True)
test_similarity(source_fast_model.wv,'hi', 'go', "FastText", src=True)

test_similarity(target_fast_model.wv,'bonjour', '.', "FastText", src=False)
test_similarity(target_fast_model.wv,'bonjour', 'cours', "FastText", src=False)

RNN

In [ ]:
# Now we can create the RNN model that will translate from english to french using one of the previous embeddings

from torch import nn
import torch.nn.functional as F

class RNN(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(RNN, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [ ]:
#From this model we can create a loss function and an optimizer

def loss_function(tag_scores, gold_tags):
    loss_function = nn.NLLLoss()
    loss = loss_function(tag_scores, gold_tags)
    return loss

In [ ]:
#Now we can train the model
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = RNN(100, 128, 100, 100).to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

n_epoch = 100

for epoch in range(n_epoch):
    for sentence, tags in data:
        sentence = torch.tensor(sentence, dtype=torch.long).to(device)
        tags = torch.tensor(tags, dtype=torch.long).to(device)

        model.zero_grad()

        tag_scores = model(sentence)

        loss = loss_function(tag_scores, tags)
        loss.backward()
        optimizer.step()
        
    #here we can use the test data to evaluate the model
    
    losses = torch.zeros(len(test_data))
    
    for sentence, tags in test_data:
        sentence = torch.tensor(sentence, dtype=torch.long).to(device)
        tags = torch.tensor(tags, dtype=torch.long).to(device)

        tag_scores = model(sentence)

        loss = loss_function(tag_scores, tags)
        
        losses.append(loss.item())
        
    print("Epoch " + str(epoch) + " : " + str(losses.mean()))
    print("Std : " + str(losses.std()))
        

    print("Epoch: {}/{}.............".format(epoch, n_epoch), end=" ")
    print("Loss: {:.4f}".format(loss.item()))

To this model we can now try to add contextual embeddings

In [ ]:
#for contextual embedding we will use BERT

import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer

#we use bert and we will train it on the data
bert_model = BertModel.from_pretrained('bert-base-uncased')
#we will use the tokenizer to tokenize the sentences
sentences = ["I love machine learning", "I love coding in python"]
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

In [ ]:
#Encoder Decoder architecture


In [ ]:
import torch
import torch.utils.data as data

# Build train and test sets from source and target tokens
def build_datasets(source, target):
    src = torch.tensor(source)
    tgt = torch.tensor(target)

    # Build full dataset
    dataset = data.TensorDataset(*(src, tgt))
    
    # Split into train and test sets
    size = len(dataset)
    train_size = int(size * 0.8)
    test_size = size - train_size
    
    return data.random_split(dataset, [train_size, test_size])

# Build train and test sets
trainset, testset = build_datasets(source_tokens, target_tokens)

In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#Encoder : 

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
MAX_LENGTH =10
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
def trainIters(encoder, decoder, n_iters = 50000, print_every=1000, plot_every=100, learning_rate=0.01):

    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        src_ex = random.choice(src)
        tgt_ex = random.choice(tgt)
        print('>', src)
        print('=', tgt)
        output_words, attentions = evaluate(encoder, decoder, src)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(src, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, tgt, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

In [ ]:
evaluateRandomly(encoder1, attn_decoder1)